# DraftFPL Database -  (RUGFPL2023)

## Tools

- Python : used to grab data from the Draft FPL API and also a github repo used to grab player ids/names and position
- PostgreSQL  stores the stats and allows functionality of querying data.

___

### Extracting from FPL API

In [10]:
import requests
import pandas as pd 
import csv
from io import StringIO
pd.set_option('display.max_columns', None)

# this git repository is used so that we can cross-reference player id's and get player position
url = 'https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/gws/merged_gw.csv'

#html get
response = requests.get(url)

# converting to string and reading the data
pos = response.text
poscsv = StringIO(pos)
pos = pd.read_csv(poscsv)

# fetching from fpl api
GAME_STATUS_URL = 'https://draft.premierleague.com/api/game'
ELEMENT_INFO_URL = 'https://draft.premierleague.com/api/bootstrap-static'
LEAGUE_DATA_URL_A = 'https://draft.premierleague.com/api/league/'
LEAGUE_DATA_URL_B = '/details'
ENTRY_PICKS_URL_A = 'https://draft.premierleague.com/api/entry/'
ENTRY_PICKS_URL_B = '/event/'
ELEMENT_STATS_URL_A = 'https://draft.premierleague.com/api/event/'
ELEMENT_STATS_URL_B = '/live'
TRANSFERS_URL_A = 'https://draft.premierleague.com/api/draft/league/'
TRANSFERS_URL_B = '/transactions'
DRAFTS_URL_A = 'https://draft.premierleague.com/api/draft/'
DRAFTS_URL_B = '/choices'

## defining functions to return as JSON from the FPL API, are able to pass our team and league ID
def get_data(url):
    response = requests.get(url)
    return response.json()

def element_info():
    return get_data(ELEMENT_INFO_URL)

def getLeagueData(leagueId):
    url = f"{LEAGUE_DATA_URL_A}{leagueId}{LEAGUE_DATA_URL_B}"
    return get_data(url)

def getEntryPicks(entryId, gameweek):
    url = f"{ENTRY_PICKS_URL_A}{entryId}{ENTRY_PICKS_URL_B}{gameweek}"
    return get_data(url)['picks']

def getElementStats(gameweek):
    url = f"{ELEMENT_STATS_URL_A}{gameweek}{ELEMENT_STATS_URL_B}"
    return get_data(url)['elements']

element_info = getElementInfo()
league_data = getLeagueData(52356)  # LEAGUE ID in ()

### Data Wrangling

To retrieve the corresponding player data, I needed to extract the player ID, first name, and second name (last name) from the dictionary. This process forms the foundation for constructing the player database, where we can merge the IDs for the player picks obtained from the function getEntryPicks to match them with the player names. This is crucial for aligning the FPL API-provided team selections with specific game weeks.

In [9]:
element_info['elements'][0].keys()

dict_keys(['id', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards', 'saves', 'threat', 'yellow_cards', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'added', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'draft_rank', 'dreamteam_count', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'in_dreamteam', 'news', 'news_added', 'news_return', 'news_updated', 'points_per_game', 'second_name', 'squad_number', 'status', 'total_points', 'web_name', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_type', 'threat_rank', 'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type', 'form_rank', 'form_rank_type', 'points_per_game_rank', 'points_per_game_rank_type', 'corners_and_indirect_freekicks_order', 'corners_and_indirect_freekicks_tex

In [4]:
# Define the output file name
output_file = 'name.csv'

# Extracting headers from the dictionary keys
headers = element_info['elements'][0].keys()

# Writing data to the CSV file
with open(output_file, 'w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=headers)
    writer.writeheader()
    for element in element_info['elements']:
        writer.writerow(element)
        
        
namedf = pd.read_csv('name.csv')
namedf = namedf[['id','first_name','second_name']]
namedf['name'] = namedf['first_name'] + ' ' + namedf['second_name']
name = pd.DataFrame(namedf)
name = name[['id','name']]

___
### Creating the SQL database

I utilized PostgreSQL to store the data, so if you are using SQLite or another database management system, you may need to import a different module.
Initially, I designed the database so that I had to input each of the 8 teams for the 20+ game weeks, which helped me determine the necessary columns. The final version you see now automates the process by populating the entire database using a function, filling it for the specified number of game weeks.

In [11]:
# connection to postgresql
import psycopg2 as pg2
# Connecting to the database
conn = pg2.connect(database='RugLeague23',
                   user='postgres',password = 'PASSWORD')
# Cursor object used to execute/fetch data from result sets
cur = conn.cursor()

In [ ]:

sqlquery = '''CREATE TABLE IF NOT EXISTS fpldb(
    id SERIAL PRIMARY KEY,
    name TEXT, 
    element FLOAT, 
    position_x FLOAT, 
    minutes FLOAT, 
    goals_scored FLOAT, 
    assists FLOAT,
    clean_sheets FLOAT, 
    goals_conceded FLOAT, 
    own_goals FLOAT, 
    penalties_saved FLOAT,
    penalties_missed FLOAT, 
    yellow_cards FLOAT, 
    red_cards FLOAT, 
    saves FLOAT, 
    bonus FLOAT,
    bps FLOAT, 
    influence FLOAT, 
    creativity FLOAT, 
    threat FLOAT, 
    ict_index FLOAT, 
    starts FLOAT,
    expected_goals FLOAT, 
    expected_assists FLOAT, 
    expected_goal_involvements FLOAT,
    expected_goals_conceded FLOAT, 
    total_points FLOAT, 
    owner TEXT,
    GW FLOAT,
    position_y TEXT,
    start FLOAT
    )
'''
cur.execute(sqlquery)
conn.commit()

___
### Filling the Database

With the utilization of a function, we can simultaneously input the game weeks and owners, allowing us to join them into one comprehensive database. This database comprises labels indicating players' owners and the game week they participated in.

In [7]:
# function that takes the inputted game week and picks out the user-selected starting 11's

def fill_database_for_week(cur, conn, owner, week):
    
    # wrangling data from the FPI to get the entry picks and outputting to a csv file.
    entry_picks = getEntryPicks(owner, week)  
    output_file = 'stats.csv'

    flattened_data = []
    for key, value in getElementStats(week).items():
        row = {'id': key}
        row.update(value['stats'])
        flattened_data.append(row)

    # extracting headers
    headers = flattened_data[0].keys()

    # Writing data to the CSV file
    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=headers)
        writer.writeheader()
        for row in flattened_data:
            writer.writerow(row)

    df = pd.read_csv('stats.csv')
    df = df.rename(columns={'id': 'element'})

    # Convert JSON to DataFrame
    picks = pd.DataFrame(entry_picks)
    picksdf = picks[['element','position']]
    
    #merged with the github database using a join that matches up player id and element so that we get 
    #player position. 
    
    merge = picksdf.merge(df, on='element', how='inner')
    merged_df = pd.merge(merge, name, left_on='element', right_on='id')

    other_columns = [col for col in merged_df.columns if col != 'name']
    merged_df = merged_df[['name'] + other_columns]
    
    #removing dreamteam column and id, since we dont need it and in sql we are going to 
    #use it as a primary key.
    
    merged_df.drop(columns=['in_dreamteam', 'id'], inplace=True)
    
    #adding columns to indicate which user selected those teams, and the selected game week.
    merged_df['owner'] = owner
    merged_df['GW'] = week

    #now we need the match up , using the same github database from before we make sure that 
    #we filter the correct GW to get the correct name and position - here we are going to use
    #name to merge two data sets together to get our final data set.
    
    POS = pos[pos['GW'] == week]

    # Select both the name and position columns from the filtered pos DataFrame
    post = POS[['name', 'position']]

    # Merge merged_df DataFrame with post DataFrame on the name column
    merged_df = pd.merge(merged_df, post, on='name', how='left')
    merged_df['start'] = merged_df['position_x'].apply(lambda x: 0 if 12 <= x <= 15 else 1)

    # Create tuples for insertion into the SQL database
    gw_tuples = merged_df.values.tolist()

    # Define the SQL query with placeholders
    gwsql_query = '''
        INSERT INTO test(name, element, position_x, minutes, goals_scored, assists,
           clean_sheets, goals_conceded, own_goals, penalties_saved,
           penalties_missed, yellow_cards, red_cards, saves, bonus,
           bps, influence, creativity, threat, ict_index, starts,
           expected_goals, expected_assists, expected_goal_involvements,
           expected_goals_conceded, total_points, owner, GW, position_y, start)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                %s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    '''

    # Execute the SQL query
    cur.executemany(gwsql_query, gw_tuples)
    conn.commit()


In [6]:
# owner id's 

owner1 = 123456
owner2 = 123456
owner3 = 123456
owner4 = 123456
owner5 = 123456
owner6 = 123456

#to iterate for all previous game weeks, we write the function into a for loop.
start_week = 1
end_week = 26
owners = [owner1,owner2,owner3,owner4,owner5,owner6] # List of owners to iterate over

for owner in owners:
    for week in range(start_week, end_week + 1):
        fill_database_for_week(cur, conn, owner, week)

___

### Building Final Standings


In [2]:
#from league_data, we grab the current standings (current gameweek).
league_data.keys()

dict_keys(['league', 'league_entries', 'matches', 'standings'])

In [3]:
leaguedf = pd.DataFrame(league_data['league_entries']) # grabbing final standings

In [5]:
standingsdf = pd.DataFrame(league_data['standings'])

In [6]:
finalstandings = pd.merge(standingsdf, leaguedf, left_on='league_entry', right_on='id')

In [11]:
finalstandings = finalstandings[['entry_name', 'matches_won', 'matches_drawn', 'matches_lost']]

finalstandings = finalstandings.rename(columns={'entry_name': 'Team',
                                                'matches_won': 'W',
                                                'matches_drawn': 'D',
                                                'matches_lost': 'L'})

finalstandings.replace({'Team': {None: 'Average'}}, inplace=True)


In [12]:
finalstandings

,Team,W,D,L
0,Porros Hermanos ⭐️,17,1,8
1,Teta Time,15,1,10
2,xG FC,14,1,11
3,JTFC,14,1,11
4,Top Boys,13,2,11
5,Quantum Boys FC,13,0,13
6,Cheffield Utd,12,2,12
7,Average,10,1,15
8,Ten Hairs,10,0,16
9,Barbican Town,7,1,18


In [13]:
# building the table from our csv data file
tsqlquery = '''CREATE TABLE IF NOT EXISTS Standings(
    Team TEXT,
    W FLOAT,
    D FLOAT,
    L FLOAT
    )
'''
cur.execute(tsqlquery)
conn.commit()

In [15]:
# Create tuples for insertion into the SQL database
st_tuples = finalstandings.values.tolist()

    # Define the SQL query with placeholders
stql_query = '''
        INSERT INTO Standings(Team,W,D,L)
        VALUES (%s,%s,%s,%s)
    '''

# Execute the SQL query
cur.executemany(stql_query, st_tuples)

conn.commit()

___
### SQL Querying in Python

If you wish to query the data in Python, you do the following below. This method is excellent for extracting relevant data by querying, and if desired, you could pipeline it into TensorFlow to construct a model.

In [12]:
cur.execute("""
            SELECT * FROM fpl23 ;
            """)
rows = cur.fetchall()

In [13]:
dffpl = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])

In [14]:
dffpl

,id,name,element,position_x,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,total_points,owner,gw,position_y,start
0,3417,Nick Pope,424.0,1.0,90.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,18.0,33.8,0.0,0.0,3.4,1.0,0.00,0.00,0.00,1.90,3.0,237040,1.0,GK,1.0
1,3418,Reece James,206.0,2.0,75.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,11.4,35.9,3.0,5.0,1.0,0.03,0.42,0.45,1.23,2.0,237040,1.0,DEF,1.0
2,3419,Ivan Perišić,507.0,3.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.4,16.3,0.0,2.5,0.0,0.00,0.16,0.16,0.30,1.0,237040,1.0,DEF,1.0
3,3420,Ezri Konsa Ngoyo,48.0,4.0,90.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,13.2,1.3,0.0,1.5,1.0,0.00,0.01,0.01,3.44,0.0,237040,1.0,DEF,1.0
4,3421,Kevin De Bruyne,349.0,5.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,2.4,18.8,1.0,2.2,1.0,0.02,0.16,0.18,0.10,1.0,237040,1.0,MID,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3370,6828,Taiwo Awoniyi,437.0,11.0,66.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,27.0,33.2,2.9,61.0,9.7,1.0,0.71,0.00,0.71,0.45,7.0,199804,25.0,FWD,1.0
3371,6829,André Onana,597.0,12.0,90.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,15.0,20.6,0.0,0.0,2.1,1.0,0.00,0.01,0.01,1.74,3.0,199804,25.0,GK,0.0
3372,6830,Nicolas Jackson,211.0,13.0,81.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,29.6,24.3,23.0,7.7,1.0,0.60,0.21,0.81,2.57,5.0,199804,25.0,FWD,0.0
3373,6831,Alexander Isak,415.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,199804,25.0,FWD,0.0


In [5]:
cur.close()